## `Практикум по программированию на языке Python`
<br>

## `Занятие 8: Обработка текстов в Python`
<br><br>

### `Находнов Максим (nakhodnov17@gmail.com)`

#### `Москва, 2022`

О чём можно узнать из этого ноутбука:

* Методы преобразования текстов в векторное представление
* Стандартный пайплайн предобработки текстов
* Базовые приёмы работы с регулярными выражениями

In [1]:
import numpy as np
np.set_printoptions(precision=4)

## `Задача обработки текстов`

Цели: 
1. Преобразование данных в формат, пригодный для использования в моделях машинного обучений — **Векторизация**
2. Использование специфики данных для улучшения качества — **Предобработка**

## `Векторизация`

Отличия текстового домена от табличных данных и изображений:
1. Нет регулярной структуры
2. Нет естественной метрики/функции расстояния

$\Longrightarrow$ попробуем преобразовать тексты в формат с которым мы уже умеем работать — **вещественные вектора**

### `Векторизация. Методы`

1. Базовые подходы:
    * **Мешок слов (Bag Of Words)**
    * **Tf-Idf**
2. Матричные разложения:
    * LSA/LDA
    * BigARTM
3. Нейросетевые подходы:
    * Word2Vec (Skip-gram, CBoW, FastText, Glove, ...)
    * BERT
    
Разберём базовые методы.

### `Формальное представление множества текстов`

$$D = \{d_1, d_2 \ldots d_N \} \; \text{— обучающая коллекция документов}$$

Обычно документы представляют в виде последовательности токенов из словаря $V$:

$$d_i = (w_1, w_2, \ldots w_{n_d}), \quad n_d \; \text{— длина документа $d$}$$

Выбор словаря и способа разбиения зависит от задачи!

### `Bag Of Words`

Предположим:
* Порядок токенов в документе не важен
* Важено лишь сколько раз токен $w$ входит в документ $d$: $\text{tf}(w, d)$

Тогда представим документ в виде вектора длины $|V|$:
$$v(d) = \{\text{tf}(w_{1}, d), ..., \text{tf}(w_{|V|}, d) \} \in \mathbb{R}^{|V|}$$

Можно выбирать разные $\text{tf}(w, d)$ в зависимости от задачи!

### `Bag Of Words. Выбор функции встречаемости` $\text{tf}$

$\text{tf}(w, d)$ — term frequency weight.

Варианты определения:
$$\text{tf}(w, d) = \mathbb{1}[w \in d]$$
$$\text{tf}(w, d) = \sum\limits_{w^{\prime} \in d} \mathbb{1}[w = w^{\prime}] = n_{wd}$$
$$\text{tf}(w, d) = \frac{\sum\limits_{w^{\prime} \in d} \mathbb{1}[w = w^{\prime}]}{\sum\limits_{w^{\prime} \in d} 1} = \frac{n_{wd}}{n_{d}}$$
$$\text{tf}(w, d) = 1 + \log(n_{wd})$$

### `Tf-Idf`

В модели BoW используется только информация о распредении слов внутри отдельных документов. Попробуем учесть распределение слов во всём корпусе текстов введя величину $\text{idf}(w)$.

Существуют разные определения, но обычно используется следующая формула:

$$idf(w) = \log \left(\frac{|D|}{\sum\limits_{d \in {D}}\mathbb{I}[w \in d]} \right) + 1$$

**Модель TF-IDF:**

Каждый документ представляется вектором длины $|V|$:
$$v(d) = \{tf(w_{1}, d) \cdot idf(w_{1}), ..., tf(w_{|V|}, d) \cdot idf(w_{|V|})\} \in \mathbb{R}^{|V|}$$

### `Реализации моделей в scikit-learn`

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
s = [
    'my name is',
    'your name are',
    'my father is'
]
vectorizer = CountVectorizer()
vectorizer.fit_transform(s).toarray()

array([[0, 0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1, 1],
       [0, 1, 1, 1, 0, 0]])

In [4]:
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(s).toarray()

array([[0.    , 0.    , 0.5774, 0.5774, 0.5774, 0.    ],
       [0.6228, 0.    , 0.    , 0.    , 0.4736, 0.6228],
       [0.    , 0.6809, 0.5179, 0.5179, 0.    , 0.    ]])

### `BoW и Tf-Idf в сравнении с более продвинутыми моделями`

| **Model**                         | **AG news** | **DBpedia** |
|-----------------------------------|-------------|-------------|
| **BoW**                           | 88.8        | 96.6        |
| **ngrams**                        | 92.0        | 98.6        |
| **ngrams TFIDF**                  | 92.4        | **98.7**        |
| **char-CNN**                      | 87.2        | 98.3        |
| **char-CRNN**                     | 91.4        | 98.6        |
| **VDCNN**                         | 91.3        | **98.7**        |
| **fastText (ngrams=2)**           | 92.5        | 98.6        |
| **StarSpace (ngrams=2)**          | **92.7**        | 98.6        |

<center><a href="https://arxiv.org/abs/1709.03856">Подробнее результаты сравнения в статье StarSpace: Embed All The Things!</a></center>


### `Преимущества и недостатки BoW и Tf-Idf`

<font color='green'> 
    Преимущества:
<ul>1. Простая и лёгкая в реализации модель </ul>
<ul>2. Довольно неплохой бейзлайн </ul>
</font>


<font color='red'> 
    Недостатки:
<ul>1. Огромная размерность признакового пространства: $|V| \approx 5 \times 10^{5}$ </ul>
<ul>2. Разреженное представление </ul>
<ul>3. Нет учёта контекста и порядка слов в предложении (без использования n-грамм) </ul>
</font>

### `Недостатки BoW и Tf-Idf. Высокая размерность`

Как бороться с огромной размерностью?

* Предобработка данных
* Методы понижения размерности (PCA)
* Использовать другие представления (word embeddings, topic modeling)

## `Предобработка текстов. Задачи`

Основная цель обработки текстов: улучшение качества работы алгоритмов машинного обучения. 

* Препроцессинг — отделение текста от "мусора"
   - `<p>Атрибут href (от англ. <i>hypertext reference</i>&nbsp;)<a href="/html/a/target">target</a>.</p>` *— теги, непечатные символы, Unicode, язык*

* Токенизация: *I'm — один токен или два?*

* Определение границ предложения: *Mr. Bing — одно предложение или два?*

* Нормализация (стемминг и лемматизация): *Красивый, красивая, красивое — разные токены?*

* Отбор признаков (токенов): *Нужны ли признаки для слов то, либо, нибудь?*

* Выделение коллокаций (n-грамм): *Метод опорных векторов — коллокация*

### `Препроцессинг. Удаление лишних символов`

Обычно, не хотим различать слова с заглавной и строчной буквами $\Rightarrow$ перед работой приводим строки в нижний регистр

In [5]:
line = 'Oh my god it is very hard'

In [6]:
line.lower()

'oh my god it is very hard'

Обычно, не хотим использовать не буквы и не цифры $\Rightarrow$ удалим все лишние символы

Воспользуемся библиотекой `regex` для работы с регулярными выражениями:

In [7]:
import regex

s = '<p>атрибут href (от англ. <i>hypertext reference</i>&nbsp;)<a href="/html/a/target">target</a>.</p>'
regex.sub('[^a-zа-яё ]', ' ', s)

' p атрибут href  от англ   i hypertext reference  i  nbsp   a href   html a target  target  a    p '

### `Препроцессинг. Удаление лишних символов`

Знаки пунктуации можно импортировать из модуля `string`. В нем хранятся различные наборы констант для работы со строками (пунктуация, алфавит и др.).

In [8]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Обратите вниманине, не всегда пунктуация является шумовым признаком! Например, в задаче sentiment analysis смайлики могут существенно влиять на качество модели.

### `Введение в регулярные выражения. Поиск совпадений`

`regex.search` может работать как `str.find`:

In [9]:
line.find('is very')

13

In [10]:
regex.search('is very', line)

<regex.Match object; span=(13, 20), match='is very'>

`regex.search` может работать как `str.startwith`:

In [11]:
line.startswith('is very')

False

In [12]:
regex.search('^is very', line)

In [13]:
regex.search('^Oh', line)

<regex.Match object; span=(0, 2), match='Oh'>

### `Введение в регулярные выражения. Основные паттерны`

- `[abcd]` — любой символ из `a, b, c, d`
- `[a-z]`— любой символ c `a` по `z`
- `[^xy]` — любой символ, не совпадающий с символами `x, y`
- `()` — скобки для выделения групп символов

In [14]:
line = 'Hello hell helio heliO'
regex.search('[Hh]el[^l][A-Z]', line)

<regex.Match object; span=(17, 22), match='heliO'>

### `Введение в регулярные выражения. Основные паттерны`

Метасимволы:
- `.` — любой символ
- `^` — начало строки
- `$` — конец строки
- `|` — оператор или
- `?` — символ перед вопросом ровно ноль или один раз
- `+` — любая ненулевая последовательность из символа перед звёздочкой
- `*` — любая последовательность из символа перед звёздочкой (в том числе и нулевой длины)

In [15]:
line = 'Oh myyy god it is 12213'
regex.search('[^a-zA-Z ]+', line)

<regex.Match object; span=(18, 23), match='12213'>

### `Введение в регулярные выражения. Основные паттерны`

Классы символов:
- `\s` — любой пробельный символ
- `\S` — любой НЕ пробельный символ
- `\p{Punct}` и другие `\p{...}` — Java расширение для выделения особых классов символов. Очень полезно для работы с Unicode. [Подробнее смотри здесь](https://www.regular-expressions.info/unicode.html#prop).

In [16]:
line = 'Oh myyy        god it is very hard'
regex.search('Oh my*\s*', line)

<regex.Match object; span=(0, 15), match='Oh myyy        '>

### `Введение в регулярные выражения. Основные функции`

`regex.compile(pattern)` — строковое представление выражения преобразуется в программное:

In [17]:
pattern = regex.compile(u'[^a-zа-яё ]+')

`regex.sub(pattern, repl, string, count=0)` — заменить `count` символов в `string`, удовлетворяющих `pattern`, на `repl`:

In [18]:
regex.sub('[^0-9]', '!', '1995 year was...', count=4)

'1995!!!!r was...'

### `Введение в регулярные выражения. Основные функции`

`regex.split(pattern, string, maxsplit=0)` — split по символам, удовлетворяющим `pattern`:

In [19]:
regex.split('[^a-z ]', "i look for. for many years. in")

['i look for', ' for many years', ' in']

`regex.findall(pattern, string)` — выделение всех непереесекающихся подстрок в строке `string`, удовлетворяющих шаблону `pattern`:

In [20]:
regex.findall('experiment_([0-9]*)_k=\(([0-9])\)', 'sometext_experiment_001_k=(5)')

[('001', '5')]

### `Регулярные выражения. Полезные ссылки`

- [Документация библиотеки `re`](https://docs.python.org/3/library/re.html)
- [Сервис онлайн проверки регулярных выражений](https://regex101.com/)
- [Упражнения на регулярные выражения](https://regexone.com/)
- ["Регулярный" кроссворд](https://mariolurig.com/crossword/)
- [Ещё хорошая справка](https://www.regular-expressions.info/quickstart.html)

## `Токенизация`

Токенизация — разделение текста на токены, элементарные единицы текста

В большинстве случае токен это слово!

Если пользоваться методом .split(), токен — последовательность букв, разделённая пробельным символам

Можно использовать регулярные выражения и модуль `regex`

Можно использовать специальные токенизаторы, например из
`nltk`:
- `RegexpTokenizer`
- `BlanklineTokenizer`
- И ещё около десятка штук

### `Библиотека nltk. Токенизаторы`

In [21]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt');
nltk.download('stopwords');

[nltk_data] Downloading package punkt to /Users/nakhodnov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nakhodnov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Встроенные токенизаторы могут быть значительно "умнее" обычного `.split`

In [22]:
example = 'Но не каждый хочет что-то исправлять:('
word_tokenize(example, language='russian')

['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять', ':', '(']

In [23]:
example.split()

['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять:(']

### `Библиотека nltk. Токенизаторы`

Так же в этой библиотеке есть более спецефичные токенизаторы:

In [24]:
nltk.tokenize.TreebankWordTokenizer().tokenize("don't stop me")

['do', "n't", 'stop', 'me']

In [25]:
nltk.tokenize.SExprTokenizer().tokenize("(a (b c)) d e (f)")

['(a (b c))', 'd', 'e', '(f)']

## `Разбиение на предложения`

Простой вариант:    

In [26]:
sent = 'Hey! Is Mr. Bing waiting for you?'
regex.split('[!.?]+', sent)

['Hey', ' Is Mr', ' Bing waiting for you', '']

Можно учитывать разные исключения при разделении:    

In [27]:
sentenceEnders = regex.compile(r"""
(?: # Group for two positive lookbehinds.
(?<=[.!?]) # Either an end of sentence punct,
| (?<=[.!?]['"]) # or end of sentence punct and quote.
) # End group of two positive lookbehinds.
(?<! Mr\. ) # Don't end sentence on "Mr."
(?<! Mrs\. ) # Don't end sentence on "Mrs."
\s+ # Split on whitespace between sentences.
""", regex.IGNORECASE | regex.VERBOSE)
sentenceEnders.split(sent)

['Hey!', 'Is Mr. Bing waiting for you?']

## `Разбиение на предложения`

Или использовать готовую реализацию из `nltk`:

In [28]:
nltk.tokenize.sent_tokenize(sent)

['Hey!', 'Is Mr. Bing waiting for you?']

## `Отбор слов`

Какие слова могут быть плохие?
* Слишком частые
    - *русский язык: и, но, я, ты, ...*
    - *английский язык: a, the, I, one, ...*
    - *специфичные для коллекции: «сообщать» в новостях*
* Слишком редкие (встречаются в $\leq 5$ документах)
* Стоп-слова (предлоги, междометия, частицы, цифры)

In [29]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
list(stopWords)[:6]

["doesn't", 'having', "weren't", "mightn't", 'own', 'o']

### `Замена сокращений с помощью regex.sub`

In [30]:
def delete_to_be(text_string):
    return regex.sub("('s|'re|n't)\s", u' <stop> ', text_string.lower())
    
delete_to_be("Where's your spoon daddy")

'where <stop> your spoon daddy'

In [31]:
delete_to_be("Why don't you like me")

'why do <stop> you like me'

### `Замена специфичных сущностей на теги`

In [32]:
sentence = """
Контактную информацию вы можете уточнить, перейдя по ссылке
https://minust.ru/structure/00000000000
(выбрать раздел &quot;Территориальные органы и подведомственные
организации&quot;, выбрать регион и открыть вкладку
&quot;Информация и контакты&quot;)
"""

In [33]:
result_string = regex.sub('&quot', ' " ', sentence)
result_string = regex.sub('(http|www)\S+', ' <URL> ', result_string)
result_string = regex.sub('([^\w\s<>])', ' \\1 ', result_string)

" ".join(result_string.split())

'Контактную информацию вы можете уточнить , перейдя по ссылке <URL> ( выбрать раздел " ; Территориальные органы и подведомственные организации " ; , выбрать регион и открыть вкладку " ; Информация и контакты " ; )'

## `Нормализация слов`

### `Стемминг`

Стемминг — отбрасывание окончаний слов

In [34]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer(language='english')
sentence = 'George admitted the talks happened'.split()
" ".join([stemmer.stem(word) for word in sentence])

'georg admit the talk happen'

In [35]:
sentence = 'write wrote written'.split()
" ".join([stemmer.stem(word) for word in sentence])

'write wrote written'

### `Стемминг для русского языка`

Для русского языка стемминг не очень подходит:

In [36]:
stemmer = SnowballStemmer(language='russian')
sentence = 'опрошенных считают налоги необходимыми'.split()
" ".join([stemmer.stem(word) for word in sentence])

'опрошен счита налог необходим'

In [37]:
sentence = 'поле пол полёт полка полк'.split()
" ".join([stemmer.stem(word) for word in sentence])

'пол пол полет полк полк'

In [38]:
sentence = 'крутой круче крутейший крутить'.split()
" ".join([stemmer.stem(word) for word in sentence])

'крут круч крут крут'

### `Вспомогательная задача — определение части речи`

In [39]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.ADJ, 'V': wordnet.VERB,
        'N': wordnet.NOUN, 'R': wordnet.ADV
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.NOUN

In [40]:
sentence = 'George admitted the talks happened'.split()
pos_taged = nltk.pos_tag(sentence)
pos_taged

[('George', 'NNP'),
 ('admitted', 'VBD'),
 ('the', 'DT'),
 ('talks', 'NNS'),
 ('happened', 'VBD')]

In [41]:
[get_wordnet_pos(tag) for word, tag in pos_taged]

['n', 'v', 'n', 'n', 'v']

### `Лемматизация`

**Лемматизация** – это сведение разных форм одного слова к начальной форме – **лемме**. Почему это хорошо?
* Во-первых, естественно рассматривать как отдельный признак каждое *слово*, а не каждую его отдельную форму.
* Во-вторых, некоторые стоп-слова стоят только в начальной форме, и без лематизации выкидываем мы только её.

Для английского можно использовать лемматизатор `WordNet`. Однако, он требует для работы метки частей речи!

### `Лемматизация`

In [42]:
from nltk import WordNetLemmatizer

def simple_lemmatizer(sentence):
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sentence.split()
    pos_taged = [
        (word, get_wordnet_pos(tag))
        for word, tag in nltk.pos_tag(tokenized_sent)
    ]
    return " ".join([
        lemmatizer.lemmatize(word, tag)
        for word, tag in pos_taged
    ])

In [43]:
simple_lemmatizer('George admitted the talks happened')

'George admit the talk happen'

In [44]:
simple_lemmatizer('write wrote written')

'write write write'

### `Лемматизация для русского языка`

Для русского есть два хороших лемматизатора: mystem и pymorphy.

[Mystem](https://tech.yandex.ru/mystem/) Как с ним работать:
* можно скачать mystem и запускать [из терминала с разными параметрами](https://tech.yandex.ru/mystem/doc/)
* [pymystem3](https://pythonhosted.org/pymystem3/pymystem3.html) - обертка для питона, работает медленнее, но это удобно

### `Лемматизация для русского языка`

In [45]:
import pymorphy2
def simple_lemmatizer(sentence):
    lemmatizer = pymorphy2.MorphAnalyzer()
    tokenized_sent = sentence.split()
    return " ".join([
        lemmatizer.parse(word)[0].normal_form
        for word in tokenized_sent
    ])

In [46]:
simple_lemmatizer('опрошенных считают налоги необходимы')

'опросить считать налог необходимый'

In [47]:
simple_lemmatizer('поле пол полёт полка полк')

'поле пол полёт полка полк'

`Pymorphy2` не требует метку части речи, но для улучшения разбора, её можно использовать.

### `Заключение о нормализации и отборе слов`

Отбор слов
* Нужен почти всегда для всех языков
* Можно сократить словарь до $100000$ токенов почти без потери качества

Стемминг
* Плохо работает для русского языка
* Нормально работает для английского, но модели хорошо работают и без него

Лемматизация
* Лучше стемминга для русского языка
* Сильно повышает качество моделей для русского языка
* Хорошо работает и для английского, но модели хорошо работают и без неё
* Гораздо медленнее чем стемминг

## `Последовательности слов`

Попробуем решить проблему BoW/Tf-Idf, учитывая не только сами токены, но и их контекст.

Рассмотрим разные сущности на примере предложения:

**<center>Метод опорных векторов — метод машинного обучения.**

* Коллокации — устойчивые словосочетания
    - `метод опорных векторов, метод машииного обучения, опорных векторов, машинного обучения`
* $n$-граммы — последовательности из n слов
    - $2$-граммы: `метод опорных, опорных векторов, векторов метод, метод машинного, машинного обучения`
* $s$-скип-$n$-граммы — последовательности из $n$ слов с $s$ пропусками
    - $1$-скип-$2$-граммы: `метод векторов, опорных метод, векторов машинного, метод обучения`

### `Выделение n-грамм`

В scikit-learn есть встроенное выделение n-грамм

In [48]:
s = ['my name is', 'your name are', 'my father is']
vectorizer = CountVectorizer(ngram_range=(1, 1))
vectorizer.fit_transform(s).toarray()

array([[0, 0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1, 1],
       [0, 1, 1, 1, 0, 0]])

In [49]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
vectorizer.fit_transform(s).toarray()

array([[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
       [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])

In [50]:
print(vectorizer.vocabulary_)

{'my': 4, 'name': 7, 'is': 3, 'my name': 6, 'name is': 9, 'your': 10, 'are': 0, 'your name': 11, 'name are': 8, 'father': 1, 'my father': 5, 'father is': 2}


### `Как можно получать коллокации`

* Извлечение биграмм на основе частот и морфологических шаблонов
* Поиск разрывных коллокаций
* Извлечение биграмм на основе мер ассоциации и статистических критериев (TopMine)
* Алгоритм `TextRank` для извлечения словосочетаний
* Rapid Automatic Keyword Extraction
* Выделение ключевых слов по Tf-Idf

### `Выделение частотных коллокаций`

Статистические алгоритмы выделения коллокаций основаны на том, что коллокацией являются слова, часто встречающиеся рядом друг с другом

**ENSURE** $\{(w, u)\}~\text{— множество коллокаций из}~2~\text{слов}$

$n_{wu}~:=~0,~\forall~w,~u~\in~V'$

**FOR** ($i=1,~\dots,~N$) **DO**

$~~~~$**FOR** ($j=1,~\dots,~n_d - 1$) **DO**

$~~~~~~~~n_{wu}~=~n_{wu}~+~\mathbb{1}[w_{j + 1}^d = w,~w_{j}^d = u]~\forall w, u \in V'$

$s~=~\{(w,u)~|~n_{wu}>t\}$

**RETURN** $s$

Без выделения стоп-слов подобные алгоритмы будут работать очень плохо!

## `Стандартные этапы обработки текстов`

Можно выделить следующие этапы:
1. Удаление специфичных символов/последовательностей
2. Приведение к нижнему регистру
3. Токенизация
4. Лемматизация
5. Выделение коллокаций
6. Удаление стоп-слов / Сокращение словаря

Почти всегда правильная обработка приводит к улучшению качества и уменьшению времени работы.

## `Спасибо за внимание!`